In [1]:
#!/home/fernandosjp/anaconda/bin/python

# Crawler
import mechanize
from BeautifulSoup import BeautifulSoup
# Visualization
from IPython.display import display, HTML
# Data
import pandas as pd
import sqlite3
# Emails
from emailSender import *
# Logging
import logging
#Others
import datetime
#Regexp 
import re 

In [2]:
#Logging
# create logger with 'alert tesouro'
logger = logging.getLogger('alert_tesouro')
logger.setLevel(logging.DEBUG)
# create file handler which logs even debug messages
fh = logging.FileHandler('alertInvest.log')
fh.setLevel(logging.DEBUG)
# create file handler which logs even debug messages
eh = logging.FileHandler('alertInvestError.log')
eh.setLevel(logging.ERROR)
# create console handler with a higher log level
ch = logging.StreamHandler()
ch.setLevel(logging.INFO)
# create formatter and add it to the handlers
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
fh.setFormatter(formatter)
ch.setFormatter(formatter)
eh.setFormatter(formatter)
# add the handlers to the logger
logger.addHandler(fh)
logger.addHandler(ch)
logger.addHandler(eh)

#Define Rules to 
alertRules = {"NTNB Princ":0.0735,
			 "LTN":0.16}

In [3]:
def convertToInt(x):
	"""
	Function to convert yield to integer
	"""
	try:
		x=float(x)/10000
	except Exception, e:
		x=0
	return x

In [4]:
def extractBondName(x):
    searchString = re.search(r"\(([^)]+)\)", str(x))
    bondName=""
    try:
        bondName = searchString.group(1)
    except:
        pass
    print bondName
    return bondName

In [5]:
def saveTask(emailSent):
	# Connecting to the database file
	sqlite_file = 'alertInvest.sqlite'  
	conn = sqlite3.connect(sqlite_file)
	c = conn.cursor()

	now = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
	
	try:
		c.execute("INSERT INTO job_logs (date_task, emailSent) VALUES (?,?)", (now,emailSent))
		logger.info('Successfully inserted in database!')
	except Exception, e:
		logger.error('Not able to INSERT in database!')

	conn.commit()
	conn.close()

	return None

In [49]:
#Open a browser
logger.info('Opening browser')
br = mechanize.Browser()
br.addheaders = [('User-agent', 'Firefox')]
br.set_handle_robots(False)

#Open a url
url='http://www.tesouro.fazenda.gov.br/tesouro-direto-precos-e-taxas-dos-titulos'
logger.info('Opening url: {}'.format(url))
page = br.open(url)

logger.info('Reading url: {}'.format(url))
html = page.read()
soup = BeautifulSoup(html)

logger.info('Getting tables from html')
tables = soup.findAll("table")

logger.info('Importing html tables to Pandas')
dfList = pd.read_html(str(tables[1]), header=False, skiprows=2)

to = ['REMOVED',
	  #'REMOVED'
]
msgText = ""
subject = "[Tesouro Direto] Alerta COMPRA TUDO - LTN >= 16%"

2015-12-20 13:38:18,265 - alert_tesouro - INFO - Opening browser
INFO:alert_tesouro:Opening browser
2015-12-20 13:38:18,298 - alert_tesouro - INFO - Opening url: http://www.tesouro.fazenda.gov.br/tesouro-direto-precos-e-taxas-dos-titulos
INFO:alert_tesouro:Opening url: http://www.tesouro.fazenda.gov.br/tesouro-direto-precos-e-taxas-dos-titulos
2015-12-20 13:38:18,615 - alert_tesouro - INFO - Reading url: http://www.tesouro.fazenda.gov.br/tesouro-direto-precos-e-taxas-dos-titulos
INFO:alert_tesouro:Reading url: http://www.tesouro.fazenda.gov.br/tesouro-direto-precos-e-taxas-dos-titulos
2015-12-20 13:38:18,807 - alert_tesouro - INFO - Getting tables from html
INFO:alert_tesouro:Getting tables from html
2015-12-20 13:38:18,809 - alert_tesouro - INFO - Importing html tables to Pandas
INFO:alert_tesouro:Importing html tables to Pandas


In [42]:
str(tables[1]).replace('rowspan="2"','').replace('colspan="2"','')

'<table width="100%" cellspacing="0" cellpadding="3" border="0"> <tbody><tr class="tabelaTitulo"> <th align="center"  class="tittuloTabelaTesouroDireto"><center><b> T\xc3\xadtulo</b></center></th> <th align="center"  class="tittuloTabelaTesouroDireto"><center><b> Vencimento</b></center></th> <!-- TD class="tabelaTitulo" rowspan=2 align=center&gt;&lt;b&gt;Indexador&lt;/b&gt;&lt;/TD --> <th  class="tittuloTabelaTesouroDireto"> <center><b>Taxa % a.a.</b></center></th> <th  class="tittuloTabelaTesouroDireto"> <center><b>Pre\xc3\xa7o Unit\xc3\xa1rio Dia</b></center></th> </tr> <tr class="tabelaTitulo1"> <th class="tittuloTabelaTesouroDireto"> <center><b>Compra</b></center></th> <th class="tittuloTabelaTesouroDireto"> <center><b>Venda</b></center></th> <th class="tittuloTabelaTesouroDireto"> <center><b>Compra</b></center></th> <th class="tittuloTabelaTesouroDireto"> <center><b>Venda</b></center></th> </tr> <tr><td bgcolor="FFFF9C" class="tittuloTabelaTesouroDireto prefixadoTesouroDireto" ali

In [50]:
dfList

[                                    Indexados ao IPCA Unnamed: 1 Unnamed: 2  \
 0      Tesouro IPCA+ com Juros Semestrais 2017 (NTNB) 2017-05-15 2015-12-20   
 1                     Tesouro IPCA+ 2019 (NTNB Princ) 2019-05-15        NaT   
 2      Tesouro IPCA+ com Juros Semestrais 2020 (NTNB) 2020-08-15        NaT   
 3      Tesouro IPCA+ com Juros Semestrais 2024 (NTNB) 2024-08-15 2015-12-20   
 4                     Tesouro IPCA+ 2024 (NTNB Princ) 2024-08-15        NaT   
 5      Tesouro IPCA+ com Juros Semestrais 2035 (NTNB) 2035-05-15        NaT   
 6                     Tesouro IPCA+ 2035 (NTNB Princ) 2035-05-15        NaT   
 7      Tesouro IPCA+ com Juros Semestrais 2045 (NTNB) 2045-05-15 2015-12-20   
 8      Tesouro IPCA+ com Juros Semestrais 2050 (NTNB) 2050-08-15        NaT   
 9                                          Prefixados        NaT        NaT   
 10                       Tesouro Prefixado 2016 (LTN) 2016-01-01 2015-12-20   
 11  Tesouro Prefixado com Juros Semestr

In [51]:
df = dfList[0]
df.dropna(inplace=True)

In [52]:
df.dtypes

Indexados ao IPCA            object
Unnamed: 1           datetime64[ns]
Unnamed: 2           datetime64[ns]
Unnamed: 3                  float64
Unnamed: 4                   object
Unnamed: 5                   object
dtype: object

In [53]:
df

,Indexados ao IPCA,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5
0,Tesouro IPCA+ com Juros Semestrais 2017 (NTNB),2017-05-15,2015-12-20,673,"R$0,00","R$2.754,32"
3,Tesouro IPCA+ com Juros Semestrais 2024 (NTNB),2024-08-15,2015-12-20,741,"R$0,00","R$2.584,85"
7,Tesouro IPCA+ com Juros Semestrais 2045 (NTNB),2045-05-15,2015-12-20,746,"R$0,00","R$2.317,43"
10,Tesouro Prefixado 2016 (LTN),2016-01-01,2015-12-20,1425,"R$0,00","R$995,77"
11,Tesouro Prefixado com Juros Semestrais 2017 (N...,2017-01-01,2015-12-20,1584,"R$0,00","R$995,47"
12,Tesouro Prefixado 2017 (LTN),2017-01-01,2015-12-20,1584,"R$0,00","R$859,74"
15,Tesouro Prefixado com Juros Semestrais 2021 (N...,2021-01-01,2015-12-20,1646,"R$0,00","R$842,85"
16,Tesouro Prefixado com Juros Semestrais 2023 (N...,2023-01-01,2015-12-20,1640,"R$0,00","R$798,31"
19,Tesouro Selic 2017 (LFT),2017-07-03,2015-12-20,2,"R$0,00","R$7.375,10"


In [8]:
logger.info('Correcting Bond Yields')
df['taxa'] = df.apply(lambda row: convertToInt(row['Compra.1']), axis=1)
df['titulo'] = df.apply(lambda row: extractBondName(row['Compra']), axis=1)

2015-12-20 02:17:44,260 - alert_tesouro - INFO - Correcting Bond Yields
INFO:alert_tesouro:Correcting Bond Yields


NTNB
NTNB
NTNB
LTN
NTNF
LTN
NTNF
NTNF
LFT


In [9]:
df

,Compra,Venda,Compra.1,Venda.1,Unnamed: 4,Unnamed: 5,taxa,titulo
1,Tesouro IPCA+ com Juros Semestrais 2017 (NTNB),2017-05-15,2015-12-20,673,"R$0,00","R$2.754,32",0,NTNB
4,Tesouro IPCA+ com Juros Semestrais 2024 (NTNB),2024-08-15,2015-12-20,741,"R$0,00","R$2.584,85",0,NTNB
8,Tesouro IPCA+ com Juros Semestrais 2045 (NTNB),2045-05-15,2015-12-20,746,"R$0,00","R$2.317,43",0,NTNB
11,Tesouro Prefixado 2016 (LTN),2016-01-01,2015-12-20,1425,"R$0,00","R$995,77",0,LTN
12,Tesouro Prefixado com Juros Semestrais 2017 (N...,2017-01-01,2015-12-20,1584,"R$0,00","R$995,47",0,NTNF
13,Tesouro Prefixado 2017 (LTN),2017-01-01,2015-12-20,1584,"R$0,00","R$859,74",0,LTN
16,Tesouro Prefixado com Juros Semestrais 2021 (N...,2021-01-01,2015-12-20,1646,"R$0,00","R$842,85",0,NTNF
17,Tesouro Prefixado com Juros Semestrais 2023 (N...,2023-01-01,2015-12-20,1640,"R$0,00","R$798,31",0,NTNF
20,Tesouro Selic 2017 (LFT),2017-07-03,2015-12-20,2,"R$0,00","R$7.375,10",0,LFT


In [10]:
rename = {'Venda':'vencimento'}
df.rename(columns=rename, inplace=True)

In [11]:
columns = ['titulo','vencimento','taxa']

logger.info(df[columns])

msgText=df[columns].to_html()

logger.info('Check condition')
emailSent = False

2015-12-20 02:17:53,513 - alert_tesouro - INFO -    titulo vencimento  taxa
1    NTNB 2017-05-15     0
4    NTNB 2024-08-15     0
8    NTNB 2045-05-15     0
11    LTN 2016-01-01     0
12   NTNF 2017-01-01     0
13    LTN 2017-01-01     0
16   NTNF 2021-01-01     0
17   NTNF 2023-01-01     0
20    LFT 2017-07-03     0
INFO:alert_tesouro:   titulo vencimento  taxa
1    NTNB 2017-05-15     0
4    NTNB 2024-08-15     0
8    NTNB 2045-05-15     0
11    LTN 2016-01-01     0
12   NTNF 2017-01-01     0
13    LTN 2017-01-01     0
16   NTNF 2021-01-01     0
17   NTNF 2023-01-01     0
20    LFT 2017-07-03     0
2015-12-20 02:17:53,559 - alert_tesouro - INFO - Check condition
INFO:alert_tesouro:Check condition
